In [9]:
def parse(path):
  g = open(path, 'r')
  for l in g:
    yield eval(l)


In [ ]:
import pandas as pd
def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('data/reviews_Grocery_and_Gourmet_Food.json')
df.head()

In [ ]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('data/reviews_Grocery_and_Gourmet_Food.json.gz')